In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
import tensorflow as tf

2025-12-14 12:45:56.802540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
import dagshub

In [17]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()


        self.score = self.model.evaluate(self.valid_generator)

        self.save_score()
        self.log_into_mlflow()  

    def save_score(self):
        scores = {
            "loss": float(self.score[0]),
            "accuracy": float(self.score[1])
        }
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        import dagshub
        import mlflow
        from mlflow.tracking import MlflowClient

        dagshub.init(
            repo_owner="reddyrohith49471",
            repo_name="Kidney-Disease-Classification-Deep-Learning-Project",
            mlflow=True
        )

        tracking_url_type_store = mlflow.get_tracking_uri().split(":")[0]

        with mlflow.start_run(run_name="model_evaluation"):
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })





In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-12-14 12:58:03,669: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-14 12:58:03,674: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-14 12:58:03,676: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.


2025-12-14 12:58:04.647968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


9/9 [==============================] - 45s 5s/step - loss: 8.4703 - accuracy: 0.5036
[2025-12-14 12:58:49,978: INFO: common: json file saved at: scores.json]
[2025-12-14 12:58:51,247: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project"

[2025-12-14 12:58:51,293: INFO: helpers: Initialized MLflow to track repo "reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project"]


...

[2025-12-14 12:58:51,299: INFO: helpers: Repository reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project initialized!]
🏃 View run model_evaluation at: https://dagshub.com/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/a25815e16a7a49b893296bae21ecc8d1
🧪 View experiment at: https://dagshub.com/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0
[2025-12-14 12:58:56,556: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project"

[2025-12-14 12:58:56,568: INFO: helpers: Initialized MLflow to track repo "reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2025-12-14 12:58:56,572: INFO: helpers: Repository reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project initialized!]
🏃 View run model_evaluation at: https://dagshub.com/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/08d810a074d14bd3a8d02e39afabc735
🧪 View experiment at: https://dagshub.com/reddyrohith49471/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0
